# PCA and Friends Session 2
This is the second session, plannend for September 24, 2021. We're going to create our perform Principal Components for real genomic data and get our feed wet with some of the standard tools.

## Prequisites
In order to follow this session, we recommend that you first install miniconda (https://docs.conda.io/en/latest/miniconda.html). Once installed, you can use our environment file to easily install all the tools you need for this session.

First, clone the github repository if you haven't done so already:

```{bash}
git clone https://github.com/stschiff/exp_dat_reading_group_2021.git
cd exp_dat_reading_group_2021
```

If you have already cloned the repository you can make sure you have the latest update by running `git pull` inside of it.

Once miniconda has been installed, you can install the environment via:

```{bash}
conda env create -f session_2/environment.yml
conda activate PCA_session_2
```
That's it. Now you should have all the tools ready.

## Getting genotype data
We are going to use Poseidon (https://poseidon-framework.github.io/#/) to easily retrieve genotype data together with some useful annotation. The tool for accessing the Poseidon package repository is named `trident`, and if you followed the recommendation for installing the conda environment above, you should have it installed already. To make sure, check with `which trident` and `trident --version`.

`trident` is a command line tool to manage Poseidon packages. Here we'll use it to automatically download packages that we need for this session. You can list all available packages like so:

In [2]:
!trident list --remote --packages

Preparing output table
found 146 packages
.-----------------------------------------.----------------.
|                  Title                  | Nr Individuals |
:=========================================:================:
| 2010_RasmussenNature                    | 1              |
| 2012_KellerNatureCommunications         | 1              |
| 2012_MeyerScience                       | 6              |
| 2012_PattersonGenetics                  | 1036           |
| 2012_PickrellNatureCommunications       | 9              |
| 2014_FuNature                           | 1              |
| 2014_GambaNatureCommunications          | 13             |
| 2014_LazaridisNature                    | 1222           |
| 2014_MalaspinasCurrentBiology           | 2              |
| 2014_OlaldeNature                       | 1              |
| 2014_RaghavanNature                     | 6              |
| 2014_RaghavanScience                    | 4              |
| 2014_RasmussenNature                    |

Here we specifically need package `2014_LazaridisNature`, which contains a lot of present-day individuals from around the world. Let's fetch that package and copy it into a local folder called `session_2/poseidon-repository` within this repository:

In [7]:
!mkdir poseidon-repository
!trident fetch -d poseidon-repository -f "*2014_LazaridisNature*"

Searching POSEIDON.yml files... 0 found
Initializing packages... 

Packages loaded: 0
Determine requested packages... 1 requested and available
Comparing local and remote package state
Handling packages
2014_LazaridisNature                    > 90.9MB to download
> 5.1% > 10.1% > 15.2% > 20.2% > 25.3% > 30.4% > 35.4% > 40.5% > 45.5% > 50.6% > 55.7% > 60.7% > 65.8% > 70.8% > 75.9% > 81.0% > 86.0% > 91.1% > 96.1% > 100.0% 


Great, now we have that package. You can checkout the files:

In [9]:
!ls poseidon-repository/2014_LazaridisNature

2014_LazaridisNature.bed   2014_LazaridisNature.fam
2014_LazaridisNature.bib   2014_LazaridisNature.janno
2014_LazaridisNature.bim   POSEIDON.yml


And you can see three genotype files (`.bed`, `.bim` and `.fam`) and an annotation file ending with `.janno`.